In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from utils import *

In [3]:
import torch
from torch import nn

In [4]:
dim = 50
max_length = 50

In [5]:
embeddings = load_glove_embeddings(dim)
len(embeddings)

400002

In [6]:
embeddings['wax']

tensor([ 0.1392,  0.7532, -0.1715, -0.6622, -0.1074,  0.3893, -0.4517, -0.7152,
        -0.3411,  0.9171,  0.5661,  0.2525,  1.0873,  0.9969,  0.1705,  0.8662,
         0.0614, -0.1375, -0.2858, -1.2783,  0.6066, -0.3275,  0.7814, -0.7399,
         0.1129, -0.3309, -1.3136,  1.4563,  0.0447, -1.1595,  0.3302, -0.5312,
        -0.6305,  0.0097, -0.5013, -0.0992,  0.1624,  0.9057,  0.7850, -0.1771,
         0.7864, -0.1800,  0.1099, -0.1697,  0.5408,  0.3937,  0.2866, -0.6887,
        -0.3778, -1.0411])

In [7]:
data = [
    ("sequencer", "dna sequencing was carried out using the dye termination kit and an automatic sequencer", "an apparatus for determining the sequence of amino acids or other monomers in a biological polymer"),
    ("order", "the templars were also known as the order of christ", "a society of knights bound by a common rule of life and having a combined military and monastic character"),
    ("horde", "an army or tribe of nomadic warriors", "the viking hordes returned to york this weekend as fierce armoured warriors mingled with the city centre crowds ."),
    ("anaemic", "although it has been thought of as a symptom of iron deficiency , it is more commonly discovered in patients who are not anemic .", "suffering from anaemia")
]

In [8]:
from torch.nn.utils.rnn import (pad_sequence, 
                                pack_padded_sequence, 
                                pad_packed_sequence)

In [9]:
sents = sorted([d[1].split() for d in data], key=lambda s: len(s), reverse=True)
sents_emb = [embeddings.sentence_to_tensor(sent) for sent in sents]
lens = [len(s) for s in sents_emb]
batch = pack_padded_sequence(pad_sequence(sents_emb), lens)

In [10]:
from wdm import LSTMEncoder, LSTMCellDecoder

In [27]:
encoder = LSTMEncoder(dim, dim)
# dim*2 because encoder is bidirectional
decoder = LSTMCellDecoder(max_length, dim*2, len(embeddings))
data_loader = init_data_loader()

In [12]:
# epochs = 100
# criterion = nn.NLLLoss()
# encoder_optim = torch.optim.Adam()
# decoder_optim = torch.optim.Adam()

# for i in range(epochs):
#     epoch_loss = 0
#     for x, y in data_loader:
#         _, (contexts, _) = encoder(x)
#         outputs = decoder(contexts)
        
#         y = pad_sequence(y, outputs.shape[0])
        
#         loss = criterion(outputs, y)
#         loss.backward()
        
#         encoder_optimizer.step()
#         decoder_optimizer.step()
        
#         epoch_loss += loss.item() / len(x)
        
#     print(f'epoch_loss={epoch_loss})

In [13]:
out, (hidden, _) = encoder(batch)

In [14]:
decoder_input = torch.cat((hidden[0], hidden[1]), dim=1)

In [24]:
out = decoder(decoder_input)

/Users/oboiko/dev/S/word_definition_modeling/wdm.py:59: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  out = F.relu(out)


In [26]:
len(out)

50

TODO: For loss function... instead of doing log_softmax, do MSE with actual GloVe vector and minimize this loss function.
Then for BLEU evaluation, you'll need a function to find the closest vector to the one produced by the model.

Interesting to compare these results to log_softmax